# Worksheet 07

Name:  Yuanyuan Wang
UID: U26234904

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.326320013253751, 5.141294243646965, 4.8018199571564955, 5.293795243018996, 5.306637742428527, 4.707778647500695, 5.209742271540497, 3.7353702380870484, 5.876773529640368, 5.084208667216384]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean1 = 8
stdev1 = 1
c2 = np.random.normal(mean1 , stdev1 , 10).tolist()
print(c2)

[8.949667056012537, 8.968662737528597, 7.770260170299656, 8.049897227585596, 6.01517330161005, 7.711751398665243, 8.840306653567517, 6.873278345963023, 7.274541548096861, 6.510801096326982]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.510801096326982, 5.084208667216384, 5.876773529640368, 7.274541548096861, 3.7353702380870484, 5.209742271540497, 4.707778647500695, 5.306637742428527, 5.293795243018996, 6.873278345963023]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.084208667216384, 3.7353702380870484, 5.209742271540497, 4.707778647500695, 5.306637742428527, 5.293795243018996]
[6.510801096326982, 5.876773529640368, 7.274541548096861, 6.873278345963023]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 4.8895888016320255,  mean_2 = 6.633848630006808
var_1 = 0.30716053787163317,  var_2 = 0.2640293518282912


/Users/wangyuanyuan/opt/anaconda3/envs/cs542/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


The mean for c1 is 5 and stdev for c1 is 1. The mean for c2 is 8 and stdev for c2 is 1.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] /  prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] /  prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  6.510801096326982
probability of observing that point if it came from cluster 0 =  1.159499795340993e-06
probability of observing that point if it came from cluster 1 =  1.3554873696065204
point =  5.084208667216384
probability of observing that point if it came from cluster 0 =  1.0625967191905694
probability of observing that point if it came from cluster 1 =  5.0013483348347743e-08
point =  5.876773529640368
probability of observing that point if it came from cluster 0 =  0.007423194181896028
probability of observing that point if it came from cluster 1 =  0.02476792050674197
point =  7.274541548096861
probability of observing that point if it came from cluster 0 =  1.052535713395418e-13
probability of observing that point if it came from cluster 1 =  0.07954512454130702
point =  3.7353702380870484
probability of observing that point if it came from cluster 0 =  0.0011151953484808564
probability of observing that point if it came from cluster 1 =  1.0234883498508462e-26
poi

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x) , sum([prob_c1_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c1_x) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6310127447960567,  P(C_2) = 0.3689872552039433
mean_1 = 4.938107744172796,  mean_2 = 6.697477423351363
var_1 = 0.3376080612538115,  var_2 = 3.3293843355241677


These values are not very different to each other and the variances are much more closer.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

6.510801096326982
Probability of coming from C_1 = 0.0003276372294729917
Probability of coming from C_2 = 0.999672362770527

5.084208667216384
Probability of coming from C_1 = 0.9452659341141126
Probability of coming from C_2 = 0.05473406588588728

5.876773529640368
Probability of coming from C_1 = 0.2670684750776249
Probability of coming from C_2 = 0.7329315249223752

7.274541548096861
Probability of coming from C_1 = 6.814683340414045e-10
Probability of coming from C_2 = 0.9999999993185317

3.7353702380870484
Probability of coming from C_1 = 0.04209562567710054
Probability of coming from C_2 = 0.9579043743228994

5.209742271540497
Probability of coming from C_1 = 0.9309503135558403
Probability of coming from C_2 = 0.06904968644415956

4.707778647500695
Probability of coming from C_1 = 0.9410923310948419
Probability of coming from C_2 = 0.058907668905158146

5.306637742428527
Probability of coming from C_1 = 0.910250018796049
Probability of coming from C_2 = 0.08974998120395099

5.293

There seem to be a lot more definitive decisions being made(probability of 1) which wasn't really happening with the kmeans implementation.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[1, 0, 1, 1, 1, 0, 0, 0, 0, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.